In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

%load_ext cython
%load_ext snakeviz

<br></br>
<br></br>

# Performance considerations in scikit-learn
---

Good Practices for efficient use of scikit-learn

<br></br>
<br></br>
<div align="center">Jérémie du Boisberranger (INRIA)</div>

intro

## Think about data types

Float32 vs float64
- 2x less memory
- faster computations (with vector instructions)
- most scikit-learn estimators support both
- numerical stability ensured by test suite

<div align='center'><img src="simd2_.png" width="900"></div>

<div align='center'><img src="simd_.png" width="700"></div>

An illustration with a simple matrix product

In [200]:
# dtype = float64
X = np.random.normal(size=(1000, 1000))
%timeit X @ X.T

14.8 ms ± 336 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [201]:
# dtype = float32
X = X.astype(np.float32)
%timeit X @ X.T

7.42 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


And using a scikit-learn estimator

In [23]:
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge

X, y = make_regression(n_samples=10000, n_features=1000)

In [8]:
%timeit Ridge().fit(X, y)

207 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
X = X.astype(np.float32)
y = y.astype(np.float32)

%timeit Ridge().fit(X, y)

110 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Type inference when reading data from file**

Example: loading data in a Pandas DataFrame

data.csv is a csv file containing the following data:

a &nbsp; b &nbsp; c  
1 &nbsp; 1 &nbsp; 1.0  
2 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.0  
3 &nbsp; 3 &nbsp; 3.0 

In [211]:
import pandas as pd

df = pd.read_csv('data.csv')
print(df)

   a    b    c
0  1  1.0  1.0
1  2  NaN  2.0
2  3  3.0  3.0


In [197]:
df.dtypes

a      int64
b    float64
c    float64
dtype: object

**Intermediate computations on the data**

Example: scipy *pdist* converts to float64

In [199]:
from scipy.spatial.distance import pdist

X = np.random.normal(size=(1000, 1000))
X = X.astype(np.float32)

pdist(X).dtype

dtype('float64')

## Dealing with large intermediate results

*Example*: K-means  
For each point in X, find its closest center in C.

Large intermediate result: pairwise distances between X and C  
`n_samples x n_clusters`

Small final result: indices of closest center for each observation  
`n_samples`

2 issues  
-> high memory consumption  
-> slower execution (data do not fit in cpu cache)

=> Compute intermediate results by chunks
<div align='center'><img src="km_perf_vs_chunk_size_.png" width="900"></div>
trade-off memory vs speed

In [213]:
from sklearn.metrics import pairwise_distances_argmin

X = np.random.random_sample((100000, 100))
C = np.random.random_sample((1000, 100))

%timeit pairwise_distances_argmin(X, C)
%memit pairwise_distances_argmin(X, C)

1.33 s ± 242 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 1594.31 MiB, increment: 762.95 MiB


In [40]:
from sklearn import config_context

with config_context(working_memory=0.1):
    %timeit pairwise_distances_argmin(X, C)
    %memit pairwise_distances_argmin(X, C)

2.37 s ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 473.14 MiB, increment: 1.09 MiB


In [219]:
with config_context(working_memory=1):
    %timeit pairwise_distances_argmin(X, C)
    %memit pairwise_distances_argmin(X, C)

1.01 s ± 212 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 831.40 MiB, increment: 0.00 MiB


## Benefit from sparsity

Datasets with lot of zeros can be stored in sparse format.

Algorithms take account of sparse representation to reduce time and memory complexity.  
e.g. `O(# elements) -> O(# non zeros)`

- Most scikit-learn estimators support sparse input.
- Preprocessing transformers return sparse output when relevant `CountVectorizer`, `OneHotEncoder`

In [4]:
from sklearn.preprocessing import OneHotEncoder

X = np.random.randint(100, size=(10000, 10))
y = np.random.randint(2, size=10000)

Xt = OneHotEncoder(categories='auto').fit_transform(X)
Xt

<10000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [5]:
# density
Xt.nnz / np.prod(Xt.shape)

0.01

In [131]:
from scipy.sparse import csr_matrix

def make_data(n_samples=100, n_features=100, density=1, random_state=0):
    rng = np.random.RandomState(random_state)
    X = rng.random_sample((n_samples, n_features))
    X[X < (1 - density)] = 0

    y = rng.randint(2, size=n_samples)
    
    return X, csr_matrix(X), y

In [205]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=10,
                           max_depth=5,
                           max_features='log2',
                           random_state=0)

In [206]:
X_dense, X_sparse, y = make_data(n_samples=10000, n_features=1000,
                                 density=0.01)

%timeit rf.fit(X_dense, y)
%timeit rf.fit(X_sparse, y)

92.2 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.4 ms ± 76.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [140]:
X_dense, X_sparse, y = make_data(n_samples=10000, n_features=1000,
                                 density=0.5)

%timeit rf.fit(X_dense, y)
%timeit rf.fit(X_sparse, y)

203 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
248 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Avoid unnecessary copy

Some data attributes can cause a copy:
- memory layout (C / Fortran)
- dtype (float32 / float64)
- sparse format (CSR / CSC)

In [156]:
from sklearn.svm import LinearSVR

X = np.random.random_sample((100000, 100))
y = np.random.random_sample(100000)

svr = LinearSVR(max_iter=20, tol=1e-16)
%memit svr.fit(X, y)

X takes 76.29 MiB in memory
peak memory: 831.14 MiB, increment: 75.88 MiB


In [148]:
X = np.asfortranarray(X)
%memit svr.fit(X, y)

peak memory: 907.47 MiB, increment: 152.22 MiB


In [158]:
X.nbytes / 2**20

76.2939453125

## Implement critical part in compiled language

- C, C++, Rust
- Cython <- scikit-learn
- Pythran
- Numba

Can be worth even when easily doable with numpy

In [8]:
X = np.random.random_sample((1000, 1000))
Y = np.random.random_sample((1000, 1000))
Z = np.random.random_sample((1000, 1000))

%timeit X + 2*Y + 3*Z

6.65 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%cython
#cython: boundscheck=False, wraparound=False
import numpy as np

def cython_sum(double[:, ::1] X, double[:, ::1] Y, double[:, ::1] Z):
    cdef:
        Py_ssize_t m = X.shape[0]
        Py_ssize_t n = X.shape[1]
        Py_ssize_t i, j
        double[:, ::1] res = np.empty((m, n))
        
    for i in range(m):
        for j in range(n):
            res[i, j] = X[i, j] + 2 * Y[i, j] + 3 * Z[i, j]
    
    return np.asarray(res)

In [9]:
%timeit cython_sum(X, Y, Z)

1.91 ms ± 92.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Parallelism

multi-processing:
    - data copy
    - inter-process communication
    - in scikit-learn: joblib (loky backend)
    
multi-threading:
    - data shared between threads
    - thread safety questions
    - in sklearn: joblib (threading backend), but GIL.
                  OpenMP in cython.

In scikit-learn: controlled by the `n_jobs` parameter



In [10]:
from sklearn.metrics import pairwise_distances

X = np.random.normal(size=(1000, 1000))

In [11]:
%timeit pairwise_distances(X, metric='manhattan', n_jobs=1)
%timeit pairwise_distances(X, metric='manhattan', n_jobs=2)
%timeit pairwise_distances(X, metric='manhattan', n_jobs=4)

690 ms ± 7.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
402 ms ± 95.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
302 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


However greater n_jobs does not always mean better performances

In [182]:
%timeit pairwise_distances(X, metric='euclidean', n_jobs=1)
%timeit pairwise_distances(X, metric='euclidean', n_jobs=2)
%timeit pairwise_distances(X, metric='euclidean', n_jobs=4)

23.1 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
101 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
117 ms ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- Thread or process creation overhead
- Already multi-threaded by BLAS (Basic Linear Algebra Subroutines)

**Risk of oversubscription**

Nested parallelism very common in scikit-learn
- Outer parallelism with joblib, controlled by `n_jobs`  
  mostly multi-processing
- Inner parallelism with BLAS (OpenBLAS / MKL), unrestricted  
  multi-threading

*Example*:  
0uter parallel loop spawns n (# cpu) processes  
Inner parallel loop spawns n (# cpu) threads  
=> n² processes/threads

An illustration with a search for best parameters of an estimator which makes lot of BLAS calls. (on a machine with 44 cpus)

First: our baseline with `n_jobs=1`

In [ ]:
In [2]: from sklearn.linear_model import LogisticRegression 
   ...: from sklearn.model_selection import GridSearchCV
   ...: from joblib import parallel_backend 
   ...:  
   ...: X = np.random.random_sample((100000, 100)) 
   ...: y = np.random.randint(2, size=100000) 
   ...:
   ...: lr = LogisticRegression(solver='lbfgs') 
   ...: param_grid = {'C': np.logspace(-1, 3, num=10)} 
   ...:  
   ...: gs = GridSearchCV(lr, param_grid=param_grid, cv=5, n_jobs=1) 
   ...:
   ...: %timeit gs.fit(X,y)                                                                                                             
27.9 s ± 369 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

`GrisSearchCV` is embarrassingly parallel.  
Let's put `n_jobs=44` to benefit from our 44 cpus.

In [ ]:
In [3]: gs = GridSearchCV(lr, param_grid=param_grid, cv=5, n_jobs=44) 
   ...:  
   ...: %timeit gs.fit(X,y)                                                                                                
29.8 s ± 302 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

No improvment ! Even slightly worse...

<img src="gs44_ob44_.png" width="800">

We can restrict the allowed number of threads for the BLAS library.  
=> Set the `OPENBLAS_NUM_THREADS` or `MKL_NUM_THREADS` environment variable.  
(Before imports)

In [ ]:
In [2]: gs = GridSearchCV(lr, param_grid=param_grid, cv=5, n_jobs=44) 
   ...:  
   ...: %timeit gs.fit(X,y)  
3.03 s ± 49.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

<img src="gs44_ob1_.png" width="800">

In the future joblib should automatically prevent oversubscription.  
=> Limit number of threads BLAS libraries can use in each sub-process.

For multi-threaded parallelism (e.g. with OpenMP in cython)  
=> Ongoing work in threadpoolctl, https://github.com/joblib/threadpoolctl.  
(inspired by SMP, https://github.com/IntelPython/smp, from Anton Malakhov)

## Take a step back


Importance of profiling: find what needs an optimization the most.  
**Optimizing is trading maintainability for performance.**
 
- Only optimize the critical part.  
  Optimized code is often more complicated and less readable.
  
  
- CPU bound vs memory bound.  
  Don't buy brand new hardware with faster memory access to multiply matrices.
  
  
- Ratio between sequential part and parallel part.  
  More cpus can only speed up the parallel part.

In [17]:
from sklearn.cluster import KMeans

X = np.random.random_sample((10000, 100))

kmeans = KMeans(n_clusters=100, n_init=1, algorithm='full')

%snakeviz -t kmeans.fit(X)

 
*** Profile stats marshalled to file '/tmp/tmpnprf5po0'. 
Opening SnakeViz in a new tab...


<img src="prof_.png" width="1600">

# Thank you for your attention

In [ ]:
Misc

-Avoid recomputing the same thing over and over

 cache functions you'll call many times with same arguments.

 Example: a pipeline with a transformer and an estimator
          in a gridsearch for the best params of the estimator
          usefull if the fit of the transformer is computionaly expensive

- 